In [ ]:
import pandas as pd
import cv2
import os
import ast 
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
files_dir='/mnt/challenge_preprocessing/files'

In [ ]:
# This function checks whether all images in a specified directory have the same shape for filtering inconsistent one.

def all_images_same_shape(path, img_range_str):
   
    global scan
    img_range = ast.literal_eval(img_range_str)
    start, end = img_range  
    
    if '/challenge1' in path:
        path = path.replace('preprocessed', 'OriginalDatauncompressed')
    else:
        path = path.replace('preprocessed', 'dataset')

    end=len(os.listdir(path))

    shapes = set()

    scan+=1
    
    for i in range(0, end):
        img_path = os.path.join(path, f"{i}.jpg")
        img = cv2.imread(img_path)
        if img is not None:
            shapes.add(img.shape)
        else:
            return False
    
    return len(shapes) == 1

In [ ]:
"""This Process a single row of the DataFrame."""

def process_row(row):
    path, img_range_str = row['Path'], row['Range']
    if all_images_same_shape(path, img_range_str):
        return row
    else:
        return None

In [ ]:
"""Filter the DataFrame using multithreading."""

def filter_df_concurrently(df):
    filtered_rows = []
    with ThreadPoolExecutor() as executor:
        # Submit all DataFrame rows for processing
        futures = [executor.submit(process_row, row) for _, row in df.iterrows()]
        
        # Process as completed
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                filtered_rows.append(result)
    
    # Create a new DataFrame from the filtered rows
    return pd.DataFrame(filtered_rows)

In [ ]:
df = pd.read_csv(os.path.join(files_dir,'la-challenge1_train_path_range_label.csv'))

scan=0
filtered_df = filter_df_concurrently(df)

shuffled_df = filtered_df.sample(frac=1).reset_index(drop=True)
shuffled_df.to_csv(os.path.join(files_dir,'filtered-la-challenge1_train_path_range_label.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(files_dir,'la-challenge1_valid_path_range_label.csv'))

scan=0
filtered_df = filter_df_concurrently(df)

shuffled_df = filtered_df.sample(frac=1).reset_index(drop=True)
shuffled_df.to_csv(os.path.join(files_dir,'filtered-la-challenge1_valid_path_range_label.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(files_dir,'la-challenge2_train_path_range_label.csv'))

scan=0
filtered_df = filter_df_concurrently(df)

shuffled_df = filtered_df.sample(frac=1).reset_index(drop=True)
shuffled_df.to_csv(os.path.join(files_dir,'filtered-la-challenge2_train_path_range_label.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(files_dir,'la-challenge2_valid_scan_path_range_label.csv'))

scan=0
filtered_df = filter_df_concurrently(df)

shuffled_df = filtered_df.sample(frac=1).reset_index(drop=True)
shuffled_df.to_csv(os.path.join(files_dir,'filtered-la-challenge2_valid_path_range_label.csv'), index=False)